### AutoEncoder

In [1]:
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
! pip install tensorflow

INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 MB 1.0 MB/s eta 0:00:0000:0100:05m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.0 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 319.5 kB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 950.6 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 1.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 1.9 MB/s eta 0:00:000

In [ ]:
model_seq = '1'
data_path = '../data/processed/'+model_seq+'/gesture_data.npz'

# Load Preprocessed Data (Assume RSSI data is preprocessed and saved)
data_file = "gesture_data.npz"
data = np.load(data_file)
X = data['data']  # RSSI sequences
y = data['labels']  # Gesture labels

# Normalize the data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Autoencoder Definition
def create_autoencoder(input_dim):
    input_layer = Input(shape=(input_dim,))
    
    # Encoder
    encoded = Dense(128, activation='relu')(input_layer)
    encoded = Dense(64, activation='relu')(encoded)
    encoded = Dense(32, activation='relu')(encoded)

    # Decoder
    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(128, activation='relu')(decoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)

    # Models
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    encoder = Model(inputs=input_layer, outputs=encoded)

    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

input_dim = X_train.shape[1]  # Number of features

# Create Autoencoder and Encoder
autoencoder, encoder = create_autoencoder(input_dim)

# Train Autoencoder
print("Training Autoencoder...")
autoencoder.fit(
    X_train, X_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, X_test),
    verbose=1
)

# Save Encoder and Scaler
encoder.save("encoder_model.h5")
joblib.dump(scaler, "scaler.pkl")

# Extract Latent Features
print("Extracting Latent Features...")
X_train_latent = encoder.predict(X_train)
X_test_latent = encoder.predict(X_test)

# Define Classifier Model
def create_classifier(input_dim, num_classes):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

num_classes = len(np.unique(y))
classifier = create_classifier(X_train_latent.shape[1], num_classes)

# Train Classifier
print("Training Classifier...")
classifier.fit(
    X_train_latent, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_latent, y_test),
    verbose=1
)

# Save Classifier
classifier.save("classifier_model.h5")

print("Autoencoder and Classifier Training Complete.")

# Inference Example
def predict_gesture(rssi_sequence):
    """Predict gesture from RSSI sequence."""
    rssi_sequence = scaler.transform(rssi_sequence.reshape(1, -1))  # Normalize
    latent_features = encoder.predict(rssi_sequence)  # Extract features
    prediction = classifier.predict(latent_features)  # Classify
    return np.argmax(prediction)

# Example Prediction
sample_rssi = X_test[0]
predicted_gesture = predict_gesture(sample_rssi)
print(f"Predicted Gesture: {predicted_gesture}, True Label: {y_test[0]}")